In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook


In [2]:
# Install tmdbsimple (only need to run once)
!pip install tqdm

In [3]:
# Load API Credentials
with open ('/Users/jhugh/.secret/yelp_api.json') as f:
    login = json.load(f)
    
#Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api_key'], timeout_s=5.0)

In [4]:
# set our API call parameters
LOCATION = 'Orlando, Florida'
TERM = 'Seafood'

In [5]:
#Specifying JSON_FILE filename
# include the search terms in the filename
JSON_FILE = "Data/results_in_progress_FL_seafood.json"
JSON_FILE

'Data/results_in_progress_FL_seafood.json'

In [6]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    
    # save an empty list
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
# If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")


[i] Data/results_in_progress_FL_seafood.json already exists.


In [7]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 0 previous results found.


In [8]:
#Use our yelp_api variable's search)query method
results = yelp_api.search_query(TERM =TERM,LOCATION=LOCATION,
                               offset=n_results)
results.keys()

ValueError: A valid location (parameter "location") or latitude/longitude combination (parameters "latitude" and "longitude") must be provided.